In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)


import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))



/kaggle/input/intentconan2/validation.csv
/kaggle/input/intentconan2/train.csv
/kaggle/input/intentconan2/test.csv


In [2]:
import pandas as pd
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, Trainer, TrainingArguments,DataCollatorForSeq2Seq
from torch.utils.data import Dataset
import random
from datasets import Dataset


In [3]:
train_path = "/kaggle/input/intentconan2/train.csv"   
test_path = "/kaggle/input/intentconan2/test.csv"     
val_path = "/kaggle/input/intentconan2/validation.csv"

df_train = pd.read_csv(train_path) 
df_test = pd.read_csv(test_path)
df_val = pd.read_csv(val_path)

print("Train Data Size:", df_train.shape)
print("Test Data Size:", df_test.shape)
print("Val data size : ", df_val.shape)

Train Data Size: (9532, 31)
Test Data Size: (2971, 32)
Val data size :  (1470, 32)


In [4]:
intent_tokens = {
    'positive': '<positive>',
    'informative': '<informative>',
    'questioning': '<questioning>',
    'denouncing': '<denouncing>'
}

In [5]:
from transformers import AutoModelForCausalLM, AutoTokenizer

model_name = "microsoft/DialoGPT-medium"
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token
model = AutoModelForCausalLM.from_pretrained(model_name)
special_tokens_dict = {'additional_special_tokens': list(intent_tokens.values())}
tokenizer.add_special_tokens(special_tokens_dict)
model.resize_token_embeddings(len(tokenizer))


tokenizer_config.json:   0%|          | 0.00/614 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/642 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/863M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

The new embeddings will be initialized from a multivariate normal distribution that has old embeddings' mean and covariance. As described in this article: https://nlp.stanford.edu/~johnhew/vocab-expansion.html. To disable this, use `mean_resizing=False`


Embedding(50261, 1024)

In [6]:
def format_function(example):
    intent_token = intent_tokens.get(example["csType"].strip().lower(), "<informative>")
    text = example["hatespeech"] + tokenizer.eos_token + intent_token + " " + example["counterspeech"] + tokenizer.eos_token
    return {"text": text}

train_dataset = Dataset.from_pandas(df_train)
val_dataset = Dataset.from_pandas(df_val)

train_dataset = train_dataset.map(format_function)
val_dataset = val_dataset.map(format_function)

# Step 6: Tokenize
def tokenize_function(example):
    encodings = tokenizer(
        example["text"],
        truncation=True,
        padding="max_length",
        max_length=256
    )
    encodings["labels"] = encodings["input_ids"].copy()  
    return encodings

train_dataset = train_dataset.map(tokenize_function, batched=True)
val_dataset = val_dataset.map(tokenize_function, batched=True)

Map:   0%|          | 0/9532 [00:00<?, ? examples/s]

Map:   0%|          | 0/1470 [00:00<?, ? examples/s]

Map:   0%|          | 0/9532 [00:00<?, ? examples/s]

Map:   0%|          | 0/1470 [00:00<?, ? examples/s]

In [7]:
import os
os.environ["WANDB_DISABLED"] = "true"

In [8]:
training_args = TrainingArguments(
    output_dir="./dialogpt_intent_cs",
    num_train_epochs=3,
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    warmup_steps=100,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=50,
    eval_strategy="epoch",
    save_strategy="epoch"
)

data_collator = DataCollatorForSeq2Seq(tokenizer, model=model, padding=True)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    data_collator=data_collator,
)

trainer.train()

Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


Epoch,Training Loss,Validation Loss
1,0.406300,0.414662
2,0.303200,0.432024
3,0.253800,0.443795


TrainOutput(global_step=14298, training_loss=0.3645724961433832, metrics={'train_runtime': 4239.4017, 'train_samples_per_second': 6.745, 'train_steps_per_second': 3.373, 'total_flos': 1.3278562530361344e+16, 'train_loss': 0.3645724961433832, 'epoch': 3.0})

In [9]:
# Example: Generate counterspeech with intent
input_hs = "Go back to your country"
intent = "<denouncing>"

input_text = input_hs + tokenizer.eos_token + intent + " "
input_ids = tokenizer.encode(input_text, return_tensors='pt')
input_ids = input_ids.to(model.device)

output = model.generate(
    input_ids,
    max_length=100,
    pad_token_id=tokenizer.eos_token_id,
    do_sample=True,
    top_k=50,
    top_p=0.95,
    temperature=0.9,
    num_return_sequences=1
)

reply = tokenizer.decode(output[0], skip_special_tokens=True)
print("Generated:", reply.replace(input_hs, "").strip())


The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


Generated: everyone deserves a chance to be treated with respect and dignity in their country, regardless of their nationality. promoting inclusivity and diversity enriches our society and fosters a better world for all.


In [10]:
import pandas as pd
from tqdm import tqdm
import torch



# Inference on test set
generated_counterspeeches = []

for i, row in tqdm(df_test.iterrows(), total=len(df_test)):
    hs = row['hatespeech']
    intent = intent_tokens.get(row['csType'].strip().lower(), '<informative>')
    
    # Prepare input
    input_text = hs + tokenizer.eos_token + intent + " "
    input_ids = tokenizer.encode(input_text, return_tensors='pt')
    input_ids = input_ids.to(model.device)
    
    # Generate response
    output_ids = model.generate(
        input_ids,
        max_length=100,
        pad_token_id=tokenizer.eos_token_id,
        do_sample=True,
        top_k=50,
        top_p=0.95,
        temperature=0.9
    )
    
    # Extract only the generated part (excluding input prompt)
    new_tokens = output_ids[0][input_ids.shape[-1]:]
    response = tokenizer.decode(new_tokens, skip_special_tokens=True).strip()
    
    generated_counterspeeches.append(response)

# Save predictions to a new DataFrame
df_test["generated_counterspeech"] = generated_counterspeeches

# Save to CSV
output_path = "/kaggle/working/generated_test_predictions.csv"
df_test.to_csv(output_path, index=False)

print(f"Saved generated counterspeeches to: {output_path}")


100%|██████████| 2971/2971 [21:50<00:00,  2.27it/s]

Saved generated counterspeeches to: /kaggle/working/generated_test_predictions.csv
